In [1]:
import io
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import urllib
import tarfile
import whisper
import torchaudio

from scipy.io import wavfile
from tqdm.notebook import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

2024-05-11 22:18:47.496240: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-11 22:18:47.496290: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# new tokenizer
from tokenizers import (decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer)
from transformers import GPT2Tokenizer, GPT2TokenizerFast, GPT2Model, GPT2LMHeadModel
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

new_tokenizer = Tokenizer(models.BPE())
new_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
trainer = trainers.BpeTrainer(vocab_size=1000, special_tokens=["<|endoftext|>"])
# train_file = 'fleurs_train.txt'
train_file = 'kathbath/kb_data_clean_wav/hindi/all_text.txt'
new_tokenizer.train([train_file], trainer=trainer)
new_tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
new_tokenizer.decoder = decoders.ByteLevel()
new_tokenizer = GPT2TokenizerFast(tokenizer_object=new_tokenizer)

new_tokenizer.save_pretrained("new_tokenizer_gpt2")
new_tokenizer

GPT2TokenizerFast(name_or_path='', vocab_size=1000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [4]:
# gpt2 tokenizer
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
print(len(gpt2_tokenizer.get_vocab()))
gpt2_tokenizer

# merge the vocabulary for the extended tokenizer
vocab_tokens = list(new_tokenizer.get_vocab())
decoded_tokens = [new_tokenizer.decoder.decode([token]) for token in vocab_tokens]
print(len(vocab_tokens), len(decoded_tokens))

50257
1000 1000


In [5]:
decoded_tokens

[' एस',
 'लभ',
 'एस',
 'आपक',
 'टव',
 'हतर',
 'मह',
 'दल',
 ' बल',
 ' सड',
 'सप',
 '�',
 'उनक',
 'वप',
 ' करक',
 ' नत',
 ' न',
 'मप',
 ' आय',
 ' छत',
 ' उ',
 'नर',
 ' मश',
 ' उपन',
 ' बदल',
 '�',
 'नई',
 'गढ',
 ' इन',
 ' हमन',
 ' असम',
 'ाे',
 ' कमल',
 'मय',
 ' गल',
 'रक',
 ' कथ',
 ' ऐत',
 'षमत',
 '�',
 'यसभ',
 ' वस',
 ' तहस',
 ' अफसर',
 'ब',
 'षम',
 'दय',
 'जह',
 'ज',
 ' आन',
 'यप',
 ' आव',
 '�',
 '�',
 'चन',
 'अक',
 'अपन',
 ' आदर',
 ' आह',
 'घर',
 'यवस',
 ' गर',
 ' जल',
 ' करव',
 'यकर',
 '�',
 'दक',
 'इफ',
 'दग',
 'रभ',
 ' गय',
 ' मद',
 ' समय',
 ' जव',
 ' अप',
 ' छह',
 ' चयन',
 '़ें',
 'मग',
 ' अक',
 'कस',
 ' रहत',
 ' दब',
 ' खत',
 ' आसप',
 ' कव',
 '�',
 'लस',
 'चक',
 'गत',
 ' मतलब',
 ' अम',
 ' बह',
 ' रस',
 ' तक',
 ' हट',
 ' चक',
 'नल',
 ' उनस',
 ' प',
 'जग',
 'जन',
 '�',
 ' ओवर',
 ' हर',
 ' कहन',
 'जद',
 'र',
 ' मन',
 'रय',
 ' शह',
 'ों',
 'धर',
 '�',
 'ग',
 'ीं',
 ' इस',
 ' आस',
 'जय',
 'छल',
 ' अत',
 'नप',
 'गलव',
 'जब',
 'रम',
 'रह',
 ' अवस',
 'कदम',
 ' खतर',
 'अन',
 'दस',
 'लक'

In [23]:
import json

# Sample list
# my_list = [1, 2, 3, 4, 5]

# File path to save JSON
file_path = "Tokenizer/output.json"

# Write list to JSON file
with open(file_path, 'w') as json_file:
    json.dump(decoded_tokens, json_file,ensure_ascii=False)

print("List saved to", file_path)

List saved to Tokenizer/output.json


In [24]:
import json

# File path to the JSON file
# file_path = "output.json"

# Read JSON file
with open(file_path, 'r',encoding='utf-8') as json_file:
    data = json.load(json_file)

# Print the data (list in this case)
print("Data read from", file_path, ":", data)

Data read from Tokenizer/output.json : [' एस', 'लभ', 'एस', 'आपक', 'टव', 'हतर', 'मह', 'दल', ' बल', ' सड', 'सप', '�', 'उनक', 'वप', ' करक', ' नत', ' न', 'मप', ' आय', ' छत', ' उ', 'नर', ' मश', ' उपन', ' बदल', '�', 'नई', 'गढ', ' इन', ' हमन', ' असम', 'ाे', ' कमल', 'मय', ' गल', 'रक', ' कथ', ' ऐत', 'षमत', '�', 'यसभ', ' वस', ' तहस', ' अफसर', 'ब', 'षम', 'दय', 'जह', 'ज', ' आन', 'यप', ' आव', '�', '�', 'चन', 'अक', 'अपन', ' आदर', ' आह', 'घर', 'यवस', ' गर', ' जल', ' करव', 'यकर', '�', 'दक', 'इफ', 'दग', 'रभ', ' गय', ' मद', ' समय', ' जव', ' अप', ' छह', ' चयन', '़ें', 'मग', ' अक', 'कस', ' रहत', ' दब', ' खत', ' आसप', ' कव', '�', 'लस', 'चक', 'गत', ' मतलब', ' अम', ' बह', ' रस', ' तक', ' हट', ' चक', 'नल', ' उनस', ' प', 'जग', 'जन', '�', ' ओवर', ' हर', ' कहन', 'जद', 'र', ' मन', 'रय', ' शह', 'ों', 'धर', '�', 'ग', 'ीं', ' इस', ' आस', 'जय', 'छल', ' अत', 'नप', 'गलव', 'जब', 'रम', 'रह', ' अवस', 'कदम', ' खतर', 'अन', 'दस', 'लक', ' हफ', 'डव', '�', '�', '�', 'इ', ' दम', ' सहय', ' एड', 'खल', ' नजर', 'धव', ' मस', 'ॉ', 'बर

In [20]:
model = whisper.load_model("medium")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Model is multilingual and has 762,321,920 parameters.


In [21]:
model.num_languages

99

In [22]:
model.decoder.token_embedding.weight

Parameter containing:
tensor([[-0.0026, -0.0016,  0.0062,  ...,  0.0007, -0.0092, -0.0009],
        [-0.0114,  0.0123,  0.0081,  ..., -0.0173, -0.0045, -0.0050],
        [ 0.0072, -0.0045, -0.0065,  ..., -0.0025,  0.0165, -0.0018],
        ...,
        [ 0.0015,  0.0010,  0.0010,  ..., -0.0027,  0.0107,  0.0038],
        [ 0.0039,  0.0015,  0.0009,  ..., -0.0072,  0.0132,  0.0083],
        [ 0.0097, -0.0006, -0.0127,  ...,  0.0075,  0.0166,  0.0010]],
       device='cuda:0', requires_grad=True)

In [23]:
model.decoder.token_embedding.weight.size()

torch.Size([51865, 1024])

In [44]:
decoded_tokens.remove(' ')

In [46]:
from whisper.tokenizer import get_tokenizer

tokenizer = get_tokenizer(multilingual=True,add_new_tokens=tuple(decoded_tokens))

already there:  <|endoftext|>
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �स
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there:  �
already there: 

In [47]:
tokenizer.encode('મને રમતો રમવી ગમે છે')

[156,
 103,
 106,
 156,
 16838,
 156,
 104,
 229,
 220,
 156,
 103,
 108,
 156,
 103,
 106,
 156,
 103,
 97,
 156,
 104,
 233,
 220,
 156,
 103,
 108,
 156,
 103,
 106,
 156,
 103,
 113,
 156,
 104,
 222,
 220,
 156,
 103,
 245,
 156,
 103,
 106,
 156,
 104,
 229,
 220,
 156,
 103,
 249,
 156,
 104,
 229]

In [48]:
tokenizer.total_token

60810

In [27]:
model.resize_token_embeddings(tokenizer.total_token)

In [28]:
model.decoder.token_embedding.weight

Parameter containing:
tensor([[-2.5883e-03, -1.6451e-03,  6.2218e-03,  ...,  6.5804e-04,
         -9.2163e-03, -9.1648e-04],
        [-1.1375e-02,  1.2329e-02,  8.0719e-03,  ..., -1.7319e-02,
         -4.5471e-03, -5.0163e-03],
        [ 7.1945e-03, -4.5166e-03, -6.4964e-03,  ..., -2.5425e-03,
          1.6510e-02, -1.7853e-03],
        ...,
        [ 6.5663e-01,  1.5870e+00,  5.3904e-01,  ...,  1.1048e+00,
          4.3386e-01, -1.0775e-01],
        [-1.7325e+00,  2.6380e+00,  1.1706e+00,  ...,  1.8303e-01,
         -1.9502e+00, -1.5623e+00],
        [-1.0974e+00,  1.2978e+00,  3.5609e-01,  ..., -7.9814e-02,
          1.2795e+00,  4.0345e-01]], device='cuda:0', requires_grad=True)

In [5]:
# options = dict(language='hi', beam_size=5, best_of=5)
# transcribe_options = dict(task="transcribe", **options)
# translate_options = dict(task="translate", **options)

In [28]:
# from whisper.tokenizer import get_tokenizer

# tokenizer = get_tokenizer(multilingual=True,add_new_tokens=tuple(['मुझे ','खेल ','खेलना ','पसंद ','મને ','રમતો ','રમવી ','ગમે ','છે.']))

# tokenizer = get_tokenizer(multilingual=True,add_new_tokens=tuple(['rajing','deep']))

# tokenizer = get_tokenizer(multilingual=True)

In [12]:
# for token in tokenizer.encoding._mergeable_ranks:
#     print(token)

In [30]:
# len(tokenizer.encoding._mergeable_ranks)

50257

In [49]:
tokenizer.encode('I like to play games')

[40, 411, 281, 862, 2813]

In [51]:
tokenizer.decode([40, 411, 281, 862, 2813])

'I like to play games'

In [52]:
hi_tokenized = tokenizer.encode('मुझे खेल खेलना पसंद है')
# hi_tokenized = tokenizer.encode('मुझे खेल खेलना पसंद है')
print(hi_tokenized)
print(len(hi_tokenized))

[54297, 54307, 59127, 52934, 52338, 52934, 54380, 52338, 52934, 59599, 59343, 53980, 52064, 54783, 59193, 52969]
16


In [53]:
print(tokenizer.decode([51865]))
print(tokenizer.decode([51865, 51866,]))
print(tokenizer.decode([51865, 51866, 51867]))
print(tokenizer.decode([51865, 51866, 51867, 51868]))
print(tokenizer.decode([51865, 51866, 51867, 51868, 44500]))
print(tokenizer.decode([51865, 51866, 51867, 51868, 44500, 43372]))

 अमर
 अमर मफ
 अमर मफओज
 अमर मफओज खटपट
 अमर मफओज खटपटह
 अमर मफओज खटपटहै


In [ ]:
# tokenizer.encode('मुझे खेल खेलना पसंद है',allowed_special="all")

In [12]:
import os
import base64

name  = 'multilingual'
vocab_path = "/raid/speech/rajgothi/whisper/whisper/assets/multilingual.tiktoken"
# vocab_path = "/raid/speech/rajgothi/whisper/whisper/assets/new.tiktoken"
ranks = {
    base64.b64decode(token): int(rank)
    for token, rank in (line.split() for line in open(vocab_path) if line)
}

In [14]:
decoded_string = b'\xe0\xaa\xb0\xe0\xaa\xae\xe0\xaa\xa4\xe0\xab\x8b'.decode('utf-8')

# Print the decoded string
print(decoded_string)

રમતો


In [1]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"

# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4")

AttributeError: module 'tiktoken' has no attribute 'encoding_for_model'

In [34]:
enc

<Encoding 'cl100k_base'>

In [18]:
import tiktoken
cl100k_base = tiktoken.get_encoding("gpt2")

# In production, load the arguments directly instead of accessing private attributes
# See openai_public.py for examples of arguments for specific encodings
enc = tiktoken.Encoding(
    # If you're changing the set of special tokens, make sure to use a different name
    # It should be clear from the name what behaviour to expect.
    name="cl100k_im",
    pat_str=cl100k_base._pat_str,
    mergeable_ranks=cl100k_base._mergeable_ranks,
    special_tokens={
        **cl100k_base._special_tokens,
        # "<|im_start|>": 100264,
        # "<|im_end|>": 100265,
        "मुझे " : 51866,
        "खेल ": 51867,
        "खेलना " : 51868,
        "पसंद ": 51869,
        "है" : 51870   
    }
)

In [23]:
# enc.encode('मुझे खेल खेलना पसंद है')

enc.encode('मुझे खेल खेलना पसंद है',allowed_special="all") #or write disallowed special...
# enc.encode('मुझे खेल खेलना पसंद है',allowed_special=set(["मुझे ","खेल ", "खेलना ","पसंद ","है"]))

[51866, 51867, 51868, 51869, 51870]

In [24]:
enc.decode([51866, 51867, 51868, 51869, 51870])

'मुझे खेल खेलना पसंद है'

In [21]:
len(enc._mergeable_ranks)

50257

In [14]:
cl100k_base._mergeable_ranks

{b'!': 0,
 b'"': 1,
 b'#': 2,
 b'$': 3,
 b'%': 4,
 b'&': 5,
 b"'": 6,
 b'(': 7,
 b')': 8,
 b'*': 9,
 b'+': 10,
 b',': 11,
 b'-': 12,
 b'.': 13,
 b'/': 14,
 b'0': 15,
 b'1': 16,
 b'2': 17,
 b'3': 18,
 b'4': 19,
 b'5': 20,
 b'6': 21,
 b'7': 22,
 b'8': 23,
 b'9': 24,
 b':': 25,
 b';': 26,
 b'<': 27,
 b'=': 28,
 b'>': 29,
 b'?': 30,
 b'@': 31,
 b'A': 32,
 b'B': 33,
 b'C': 34,
 b'D': 35,
 b'E': 36,
 b'F': 37,
 b'G': 38,
 b'H': 39,
 b'I': 40,
 b'J': 41,
 b'K': 42,
 b'L': 43,
 b'M': 44,
 b'N': 45,
 b'O': 46,
 b'P': 47,
 b'Q': 48,
 b'R': 49,
 b'S': 50,
 b'T': 51,
 b'U': 52,
 b'V': 53,
 b'W': 54,
 b'X': 55,
 b'Y': 56,
 b'Z': 57,
 b'[': 58,
 b'\\': 59,
 b']': 60,
 b'^': 61,
 b'_': 62,
 b'`': 63,
 b'a': 64,
 b'b': 65,
 b'c': 66,
 b'd': 67,
 b'e': 68,
 b'f': 69,
 b'g': 70,
 b'h': 71,
 b'i': 72,
 b'j': 73,
 b'k': 74,
 b'l': 75,
 b'm': 76,
 b'n': 77,
 b'o': 78,
 b'p': 79,
 b'q': 80,
 b'r': 81,
 b's': 82,
 b't': 83,
 b'u': 84,
 b'v': 85,
 b'w': 86,
 b'x': 87,
 b'y': 88,
 b'z': 89,
 b'{': 90,
 b'|': 9

In [15]:
cl100k_base._mergeable_ranks[b'rajing'] = len(cl100k_base._mergeable_ranks)

In [16]:
len(cl100k_base._mergeable_ranks)

50257

In [17]:
cl100k_base._mergeable_ranks[b'rajing']

50256

In [36]:
from transformers import WhisperTokenizer, WhisperForConditionalGeneration

# load pre-trained tokenizer and model
ckpt = "openai/whisper-small.en"
tokenizer = WhisperTokenizer.from_pretrained(ckpt)
model = WhisperForConditionalGeneration.from_pretrained(ckpt)

# define new tokens to add to vocab
new_tokens = ["मुझे","खेल", "खेलना","पसंद","है"]

# check if the new tokens are already in the vocabulary
# new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())

# add the tokens to the tokenizer vocabulary
tokenizer.add_tokens(list(new_tokens))

# add new random embeddings for the appended tokens
model.resize_token_embeddings(len(tokenizer))

Embedding(51869, 768)

In [37]:
tokenizer

WhisperTokenizer(name_or_path='openai/whisper-small.en', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|startoftranscript|>', '<|en|>', '<|zh|>', '<|de|>', '<|es|>', '<|ru|>', '<|ko|>', '<|fr|>', '<|ja|>', '<|pt|>', '<|tr|>', '<|pl|>', '<|ca|>', '<|nl|>', '<|ar|>', '<|sv|>', '<|it|>', '<|id|>', '<|hi|>', '<|fi|>', '<|vi|>', '<|iw|>', '<|uk|>', '<|el|>', '<|ms|>', '<|cs|>', '<|ro|>', '<|da|>', '<|hu|>', '<|ta|>', '<|no|>', '<|th|>', '<|ur|>', '<|hr|>', '<|bg|>', '<|lt|>', '<|la|>', '<|mi|>', '<|ml|>', '<|cy|>', '<|sk|>', '<|te|>', '<|fa|>', '<|lv|>', '<|bn|>', '<|sr|>', '<|az|>', '<|sl|>', '<|kn|>', '<|et|>', '<|mk|>', '<|br|>', '<|eu|>', '<|is|>', '<|hy|>', '<|ne|>', '<|mn|>', '<|bs|>', '<|

In [34]:
enc = tokenizer.encode(text='SDR')
enc

[50257, 50362, 50, 7707, 50256]

In [31]:
tokenizer.decode(enc)

'<|startoftranscript|><|notimestamps|>SDR<|endoftext|>'

In [29]:
enc = tokenizer.encode(text='मुझे खेल खेलना पसंद है')
enc

[50257,
 50362,
 11976,
 106,
 24231,
 223,
 11976,
 251,
 24231,
 229,
 28225,
 244,
 24231,
 229,
 11976,
 110,
 28225,
 244,
 24231,
 229,
 11976,
 110,
 11976,
 101,
 48077,
 28225,
 103,
 11976,
 116,
 11976,
 224,
 11976,
 99,
 28225,
 117,
 24231,
 230,
 50256]

In [38]:
enc = tokenizer.encode(text='मुझे खेल खेलना पसंद है')
enc

[50257, 50362, 51864, 51865, 51866, 51867, 51868, 50256]